In [1]:
# prompt: Create a function in python to greab the last 100 tweets from twitter on crypto

import tweepy

def get_last_100_tweets_on_crypto():
  # Create a Twitter API client
  api = tweepy.Client(bearer_token="YOUR_BEARER_TOKEN")

  # Get the last 100 tweets that mention "crypto"
  tweets = api.search_recent_tweets(query="crypto", tweet_fields=["text"])

  # Return the tweets
  return tweets


In [2]:
Create a function in python to greab the last 100 tweets from twitter on crypto


SyntaxError: ignored

# Building a Generative AI Application with Vertex AI and SingleStore

Welcome to this comprehensive guide on building a state-of-the-art General AI application using Google Cloud's Vertex AI and SingleStoreDB. This guide aims to provide a seamless experience, offering step-by-step instructions, code explanations, and best practices.

## Overview
Vertex AI, a product by Google Cloud, offers an integrated suite of machine learning tools that allows developers to build, deploy, and scale AI models faster than ever. On the other hand, SingleStoreDB offers a fast, scalable, and SQL-compliant relational database system. By combining the power of Vertex AI's machine learning capabilities with the efficient storage and retrieval mechanisms of SingleStoreDB, we can create robust AI applications that respond to user queries in real-time.

## What You'll Learn
- Setting up your environment with the necessary packages and credentials.
- Fetching and processing data to be used in our AI models.
- Storing and managing data efficiently using SingleStoreDB.
- Leveraging the power of Vertex AI for real-time data processing and insights.
- Building a retrieval-based QA system to answer user queries.

## Prerequisites
- Basic knowledge of Python programming.
- Familiarity with Google Cloud services and SQL databases.
- An active Google Cloud account.
- A SingleStoreDB hosted or self-managed instance.

Let's dive in and start building!



####**Setting up the environment**:
Install the required libraries.

**IMPORTANT**
Restart the runtime as prompted

In [ ]:
!pip install gcloud
!pip install langchain
!pip install singlestoredb
!pip install shapely==1.8.5
!pip install pypdf
!pip install google-cloud-aiplatform


####**Authentication**:
The next step involves authenticating our session with Google Cloud. By running the following cell, you'll be prompted to log in using your Google Cloud credentials. Follow the instructions to complete the login process.

In [ ]:
!gcloud auth application-default login

Alternative method to authenticate

In [ ]:
#from google.colab import auth as google_auth
#google_auth.authenticate_user()

#from google.cloud import aiplatform
#aiplatform.init(project="isv-coe-denisj-00")

**Setting the Quota Project**: After authentication, we need to set our quota project. Replace my-project-1516239077425 with your project ID if different, then run the cell to set the quota project for this session.

In [ ]:
!gcloud auth application-default set-quota-project cpe-ai-build

In [ ]:
!gcloud config set project cpe-ai-build

####**Import modules, initialize, load/chunk data**

**Importing Necessary Modules**: With the initial setup complete, let's import the essential classes and modules we'll use throughout this project. The following cell imports the required classes from langchain and SingleStoreDB.

In [ ]:
from langchain.llms import VertexAI
from langchain.chains import RetrievalQA
from langchain.vectorstores import SingleStoreDB

**Initializing Vertex AI**: To interact with Google Cloud's Vertex AI services, we first need to instantiate the VertexAI class. Running the cell below will create this instance and store it in the variable llm.

By default this will use text-bison model. So specify a different model use the following example:
`llm = VertexAI(model_name="code-bison")`

In [ ]:
llm = VertexAI(model_name="text-bison", max_output_tokens=2048)

**Loading Data from the Web**: Our application requires data to process and generate insights. In this step, we'll fetch content from a URL using the WebBaseLoader class. The loaded data will be stored in the data variable. You can replace the URL with any other source if needed.


In [ ]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://cloud.google.com/vertex-ai/docs/generative-ai/learn/generative-ai-studio")
data = loader.load()

**Loading Data from the PDF**: In this step, we'll fetch content from a PDF file stored locally on the system using the PyPDFLoader class. The loaded data will be stored in the data variable.

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("/home/NBA-CBA-6-28-23.pdf" )
pages = loader.load_and_split()

In [ ]:
pages[1]

**Splitting the Data**: To process the data more efficiently, we'll split the loaded content into smaller chunks. The RecursiveCharacterTextSplitter class helps in achieving this by dividing the data based on specified character limits.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 4096, chunk_overlap = 410)
all_splits = text_splitter.split_documents(pages)

####**Setting Up SingleStoreDB with Vertex AI Embeddings**:
For efficient storage and retrieval of our data, we use SingleStoreDB in conjunction with Vertex AI embeddings. The following cell sets up the necessary environment variables and initializes the SingleStoreDB instance with Vertex AI embeddings. Ensure you have the correct SingleStoreDB URL and credentials set.


In [ ]:
from langchain.embeddings import VertexAIEmbeddings
from langchain.prompts import PromptTemplate
import os

os.environ["SINGLESTOREDB_URL"] = "admin:{your admin password}.{your singlestore host}:3306/{your database name}"

vectorstore = SingleStoreDB.from_documents(documents=all_splits, embedding=VertexAIEmbeddings())
#vectorstore = SingleStoreDB(embedding=VertexAIEmbeddings())

In [ ]:
retriever=vectorstore.as_retriever(search_type="similarity")

template = """Use the following pieces of context about the NBA CBA to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Be verbose and explain your answer in simple english.
Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

####**Setting Up and Testing the QA Chain**:
Once our data is processed and stored, we can use it to answer queries. The following cell initializes the RetrievalQA chain using the previously set up llm and vectorstore. After initializing, it tests the setup with a sample question about Vertex AI.



In [ ]:
qa_chain = RetrievalQA.from_chain_type(llm,retriever=retriever,return_source_documents=1, chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})
qa_chain({"query": "How many games does an NBA player have to play in order to be eligible to win MVP?"})

In [ ]:
qa_chain({"query": "What is the best database company?"})

In [ ]:
qa_chain({"query": "What is the NBA policy on meal expenses this year?"})

In [ ]:
qa_chain({"query": "Do NBA players get 401k matching, provide details?"})

In [ ]:
qa_chain({"query": "Explain details about how the In-season tournament will be played"})

In [ ]:
qa_chain({"query": "How much is NBA player pension?"})

In [ ]:
qa_chain({"query":"What does the runner up of the in-Season Tournament get"})

In [ ]:
qa_chain({"query":"What incentive do teams have for participating in the in-Season Tournament?"})

In [ ]:
qa_chain({"query": "What is the maxium increase in salary for veteran extensions"})

In [ ]:
qa_chain({"query": "What is the maxium allowed increase in league salary cap?"})